# Retrieve spectra using Simple Spectral Access protocol

This notebook is one of a set produced by NAVO to demonstrate data access with python tools. 

In this notebook, we show how to search for and retrieve spectra from VO services using the Registry and the __[Simple Spectral Access](http://www.ivoa.net/documents/SSA/)__ (SSA) protocol.


In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline  
import requests, io
from astropy.table import Table
import astropy.io.fits as fits

import pyvo as vo

from astropy.coordinates import SkyCoord

## There are a number of relatively unimportant warnings that 
## show up, so for now, suppress them:
import warnings
warnings.filterwarnings("ignore")

First, we find out what spectral acess services ('ssa') are available in the Registry offering x-ray data:

In [ ]:
services=vo.regsearch(servicetype='ssa',waveband='x-ray')
services.table['ivoid','short_name']

We can look at only the Chandra entry:

In [ ]:
chandra_service=services[int(np.where(np.isin(services.table['short_name'],b'Chandra'))[0][0])]


## Chandra Spectrum of the Delta Ori

In [ ]:
delori=SkyCoord.from_name("delta ori")
spec_tables=chandra_service.search(pos=delori,radius=0.1)
spec_tables.table

#spec_tables[0]['time'].format='.4f'
#spec_tables[0].show_in_notebook()



In [ ]:
hdu_list=fits.open(spec_tables[0].getdataurl())

## Simple example of plotting a spectrum

In [ ]:
spec_table=Table(hdu_list[1].data)
spec_table

In [ ]:
for i in range(len(spec_table)):
    matplotlib.rcParams['figure.figsize'] = (10, 8)
    ax=plt.subplot(6,2,i+1)
    pha = plt.plot( spec_table['CHANNEL'][i],spec_table['COUNTS'][i])
    ax.set_yscale('log')
    if spec_table['TG_PART'][i] == 1:
        instr='HEG'
    if spec_table['TG_PART'][i] == 2:
        instr='MEG'
    if spec_table['TG_PART'][i] == 3:
        instr='LEG'
    ax.set_title("{grating}{order:+d}".format(grating=instr, order=spec_table['TG_M'][i]))
    plt.tight_layout()


This can then be analyzed in your favorite spectral analysis tool, e.g., [pyXspec](https://heasarc.gsfc.nasa.gov/xanadu/xspec/python/html/index.html).  (For the winter 2018 AAS workshop, we demonstrated this in a [notebook](https://github.com/NASA-NAVO/aas_workshop_2018/blob/master/heasarc/heasarc_Spectral_Access.ipynb) that you can consult for how to use pyXspec, but the pyXspec documentation will have more information.)  